In [2]:
from siftdetector import detect_keypoints
%matplotlib inline
from numpy_sift import SIFTDescriptor 
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/ubuntu/dev/opencv-3.1/build/lib')
import cv2
import time
import numpy as np
import cv2
from sklearn.cluster import KMeans
import os
from math import log
from tqdm import tqdm

# 3. Finding Interest Point Operators

Interest point operators can help us find points or features in the image that are unique and tell us something about the picture as a whole. These features are often found on the edges and corners of the image because these boundary points separate out the foreground, where interesting features are, from the lighter image background. We will be exploring a technique called MSER in this mini problem set to find high contrast regions of the image which represent edges, and sample features from those points.

## Thresholding

We will explore the concept of thresholding, which is the idea that a threshold is set between 0 and 255 (the values that a pixel can be) and all the pixels above that number are white and all those below the number are black. In the code below, the function MSER takes in a threshold value. Then the display image (a picture of a horse) is read as a numpy array with dimensions 128x128 with pixel values between 0 and 255. Your task is to fill in the code in the area specified so that pixel values below the threshold are 0, which means they are black and pixel values above the threshold are 255, which means they are white. Your final image should be called 'final' in order to be properly displayed. You can check yourself by ensuring that your pictures at threshold levels 80, 100 and 120 look like the following pictures respectively.

<img src="80" width="200" height="200"/>
<img src="100" width="200" height="200"/>
<img src="120" width="200" height="200"/>

In [ ]:
def MSER(threshold):
    display_img = cv2.imread('00000019.jpg', 0)
    #Fill in code here 

    
    black = np.asarray([[255 for _ in range(128)] for _ in range(128)])
    higher = display_img < threshold
    higher = higher.astype(int)
    final = higher * black
    
    
    plt.imshow(final, cmap='Greys'),plt.show()
    return final

#You can change the threshold to whatever you want for testing but make sure 
#to change it back to 80 before you move on to the next part
final = MSER(80)

## Picking out Features of Interest

Now that we have the outline of the feature of interest, we want to find interesting points at the boundaries. We have implemented the code to find intensity changes in the horizontal x direction but you will need to add the code to find intensity changes in the vertical y direction and plot those points.

In [ ]:
def returnPoints(picture):
    display_x_img = cv2.imread('00000019.jpg', 0)
    points = set()
    
    #code to find intensity changes in the horizontal x direction
    for x in range(len(picture)):
        for y in range(1, len(picture[0])):
            if abs(picture[x][y] - picture[x][y-1]) > 254:
                points.add((x, y))

    
    #Add your code here to add the (x,y) coordinates of points that represent an intensity change 
    #in the y direction. The points should be added to the set points.

    
    for i in points:
        y,x = i
        cv2.circle(display_x_img,(x,y),1,255,-1)
    
    plt.imshow(display_x_img),plt.show()



returnPoints(final)

display_img = cv2.imread('reject/00000002.jpg', 0)
black = np.asarray([[255 for _ in range(128)] for _ in range(128)])
print display_img[0][127]
print display_img[127][0]
def thresholding(threshold):
    higher = display_img < threshold
    print display_img
    print higher
    print higher.astype(int)
    higher = higher.astype(int)
    final = higher * black
    print "final", final
    plt.imshow(final, cmap='Greys'),plt.show()
    return final
    
final = thresholding(100)

def find_point(picture, value):
    points = []
    for x in range(len(picture)):
        for y in range(1, len(picture[0])):
            if abs(picture[x][y] - picture[x][y-1]) > value:
                points.append((x, y))
    return points

display_points = find_point(final, 256)

print display_points
#display_points = [(0, 127)]
for i in display_points:
    y,x = i
    cv2.circle(display_img,(x,y),1,255,-1)
    
plt.imshow(display_img),plt.show()

In [ ]:
def returnPointsXdirection(picture):
    display_x_img = cv2.imread('00000019.jpg', 0)
    points = set()
    for x in range(len(picture)):
        for y in range(1, len(picture[0])):
            if abs(picture[x][y] - picture[x][y-1]) > 254:
                points.add((x, y))

    #add your code here
    for y in range(128):
        for x in range(1, 128):
            if abs(picture[x][y] - picture[x-1][y]) > 254:
                points.add((x, y))

    
    for i in points:
        y,x = i
        cv2.circle(display_x_img,(x,y),1,255,-1)
    
    plt.imshow(display_x_img),plt.show()



returnPointsXdirection(final)

In [ ]:
def MSER(threshold):
    display_img = cv2.imread('00000019.jpg', 0)
    #Fill in code here 

    
    black = np.asarray([[255 for _ in range(128)] for _ in range(128)])
    higher = display_img < threshold
    higher = higher.astype(int)
    final = higher * black
    
    
    plt.imshow(final, cmap='Greys'),plt.show()
    return final

#You can change the threshold to whatever you want for testing but make sure 
#to change it back to 80 before you move on to the next part
final = MSER(80)

In [18]:
def return_labels(directoryList, range_size, added_limit, count_limit, patch_size=32):
    SD = SIFTDescriptor(patchSize = patch_size)

    #creates initial vector of size 3968
    sift_pictures = np.asarray([[0 for _ in range(range_size)]])
    for directory in directoryList:
        #go through the directories with the relevant images
        for filename in os.listdir(directory):
            name = directory + '/' + filename
            print name
            if name[-3:] == 'png' or name[-3:] == 'jpg':
                image = cv2.imread(name)
                
                #transform it to black and white and get features
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                h, w = gray.shape
                corners = cv2.goodFeaturesToTrack(gray,count_limit,0.001,10)

                corners = np.int0(corners)
                image_sift_features = np.asarray([]) 
                count, actually_added = 0, 0
                
                #Here we want to get 31 images for our feature vector and then break out of the loop
                while actually_added < added_limit and count < count_limit:
                    corner = corners[count][0]
                    if patch_size/2 <= corner[1] <= h-patch_size/2 and patch_size/2 <= corner[0] <= w-patch_size/2:
                        patch = gray[corner[1]-patch_size/2:corner[1]+patch_size/2, corner[0]-patch_size/2:corner[0]+patch_size/2]
                        sift = SD.describe(patch)
                        image_sift_features = np.append(image_sift_features, sift)
                        actually_added += 1
                    count += 1

                sift_pictures = np.append(sift_pictures, [image_sift_features], axis=0)

    return sift_pictures[1:]



In [22]:
sift_pictures = return_labels(['Auditorium', 'Bowling'], 2304, 18, 100)
kmeans = KMeans(n_clusters=2, random_state=1).fit(sift_pictures)
test_pictures = return_labels(['test'], 3968, 31, 100)
print kmeans.predict(sift_pictures)
print kmeans.predict(test_pictures)

Auditorium/.DS_Store
Auditorium/00000001.jpg
Auditorium/00000003.jpg
Auditorium/00000004.jpg
Auditorium/00000008.jpg
Auditorium/00000009.jpg
Auditorium/00000011.jpg
Auditorium/00000016.jpg
Auditorium/00000022.jpg
Auditorium/00000024.jpg
Auditorium/00000025.jpg
Auditorium/00000026.jpg
Auditorium/00000030.jpg
Auditorium/00000031.jpg
Auditorium/00000032.jpg
Auditorium/00000049.jpg
Auditorium/00000062.jpg
Bowling/.DS_Store
Bowling/00000006.jpg
Bowling/00000010(1).jpg
Bowling/00000011.jpg
Bowling/00000012(1).jpg
Bowling/00000026(1).jpg
Bowling/00000029(1).jpg
Bowling/00000031.jpg
Bowling/00000044.jpg
Bowling/00000055(1).jpg
Bowling/00000058.jpg
Bowling/00000059.jpg
Bowling/00000072.jpg
Bowling/00000095.jpg
Bowling/00000098.jpg
Bowling/00000104.jpg
Bowling/00000126(1).jpg
Bowling/00000129(1).jpg
Bowling/00000134(1).jpg
test/.DS_Store
test/00000002.jpg
test/00000017.jpg
test/00000162.jpg
test/00000190.jpg
test/00000510.jpg
test/00000591.jpg
[1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1

ValueError: Incorrect number of features. Got 3968 features, expected 2304